In [1]:
import matplotlib.pyplot as plt
import os
import importlib
import pandas

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import sys
from pathlib import Path

# Add parent directory to sys.path
parent_dir = Path("..").resolve()
if str(parent_dir) not in sys.path:
    sys.path.insert(0, str(parent_dir))
import itertools
from sklearn.metrics import confusion_matrix
import common_utils
importlib.reload(common_utils)
from common_utils import *
from tensorflow.keras.callbacks import ModelCheckpoint,ReduceLROnPlateau, EarlyStopping



In [42]:

image_paths_csv = pd.read_csv("./processed_data/image_paths_7_classes.csv")
#aug data
image_paths_csv_aug = pd.read_csv("./processed_data/image_paths_7_classes_aug.csv")

# bloack for training
train_block_1, val_block_1, train_block_2, val_block_2, test_block=split_blocks(image_paths_csv)

In [43]:
len(train_block_1), len(train_block_2), len(val_block_1), len(val_block_2), len(test_block)

(3445, 3446, 862, 862, 957)

In [44]:
train_block_1_aug=add_augmentations(train_block_1)
train_block_2_aug=add_augmentations(train_block_2)

In [19]:
# normal data

train_block_1_dataset = load_images_from_paths(paths_tensor=tf.constant(list(train_block_1_aug['path'])),batch_size=512,task="autoencoder",resize=192)

val_block_1_dataset= load_images_from_paths(paths_tensor=tf.constant(list(val_block_1['path'])),batch_size=512,task="autoencoder",resize=192)

train_block_2_dataset = load_images_from_paths(paths_tensor=tf.constant(list(train_block_2_aug['path'])),batch_size=512,task="autoencoder",resize=192)

val_block_2_dataset= load_images_from_paths(paths_tensor=tf.constant(list(val_block_2['path'])),batch_size=512,task="autoencoder",resize=192)

test_block_dataset = load_images_from_paths(paths_tensor=tf.constant(list(test_block['path'])),batch_size=512,task="autoencoder",resize=192)



NameError: name 'train_block_1' is not defined

In [2]:
configs=[
    {"name": "B2B_deep_bn_reg_aug_autoencoder","epochs":500, "l2_reg": 1e-4,"batch_size":256,"batch_norm_dense":True, "conv_filters": [32,64,128,256,512], "dense_units": [512,256,128], "batch_norm": True, "aug_data": True,"task":"autoencoder","resize":200},
]


default_args = {
    "batch_size": 512,
    "epochs": 200,
    "channels": 3,
    "dropout_rate": 0.5,
    "task": "regression",
    "num_classes": 7,
    "conv_filters": [32, 64, 128],
    "kernel_size": 3,
    "activation": "relu",
    "dense_units": [128],
    "output_activation": "softmax",
    "batch_norm" : False,
    "batch_norm_dense": False,
    "normalize" : True,
    "aug_data": False,
    "use_skip": False,
    "l2_reg":0
    
}

ratio=1
# Build all models and store in DataFrame
model_list = []
for cfg in configs:
    cfg_full = {**default_args, **cfg}
    cfg_full["loss"]='categorical_crossentropy' if cfg_full["task"] =="classification" else "mse"
    cfg_full["metrics"]=['accuracy'] if cfg_full["task"] =="classification" else ["mse","mae"]
    
    model = build_autoencoder(
            input_shape=(192, 192, cfg_full["channels"]),
            conv_filters=cfg_full["conv_filters"],
            kernel_size=cfg_full["kernel_size"],
            activation=cfg_full["activation"],
            batch_norm=cfg_full["batch_norm"],
            l2_reg=cfg_full["l2_reg"],
    )
    model.compile(optimizer='adam', loss='mse', metrics=['mse', 'mae'])
    cfg_full["model"] = model
    
    model_list.append(cfg_full)


In [49]:
config_count=0

In [3]:
model_list[0]["model"].summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 192, 192, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 192, 192, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 192, 192, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ average_pooling2d               │ (None, 96, 96, 32)     │             0 │
│ (AveragePooling2D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 96, 96, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 96, 96, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ average_pooling2d_1             │ (None, 48, 48, 64)     │             0 │
│ (AveragePooling2D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 48, 48, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 48, 48, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ average_pooling2d_2             │ (None, 24, 24, 128)    │             0 │
│ (AveragePooling2D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 24, 24, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 24, 24, 256)    │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ average_pooling2d_3             │ (None, 12, 12, 256)    │             0 │
│ (AveragePooling2D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 12, 12, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 12, 12, 512)    │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ average_pooling2d_4             │ (None, 6, 6, 512)      │             0 │
│ (AveragePooling2D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ encoded (AveragePooling2D)      │ (None, 3, 3, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_transpose                │ (None, 6, 6, 512)      │     2,359,808 │
│ (Conv2DTranspose)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_5           │ (None, 6, 6, 512)      │         2,048 │
│ (BatchNormalization)            │                        │             

 Total params: 5,504,387 (21.00 MB)

 Trainable params: 5,500,419 (20.98 MB)

 Non-trainable params: 3,968 (15.50 KB)

In [ ]:
os.makedirs(f"{configs[0]['task']}_saved_models_config_{config_count}",exist_ok=True)
for entry in model_list:
    model_name= entry["name"]
    checkpoint_path = f"{entry['task']}_saved_models_config_{config_count}/{model_name}_best_model.keras"
    checkpoint = ModelCheckpoint(
        filepath=checkpoint_path,
        monitor='val_loss',
        save_best_only=True,
        save_weights_only=False,
        verbose=0
    )

    callbacks = [checkpoint]


    early_stop_loss = DelayedEarlyStopping(
        monitor='val_loss',
        patience=10,
        verbose=0,
        restore_best_weights=False,
        delay=60  # begin checks after 60 epochs
    )
    
    callbacks.append(early_stop_loss)
    
    callbacks.append(LiveLossPlot(model_name=model_name))

    history = entry["model"].fit(
        train_block_1_dataset,
        validation_data=val_block_1_dataset,
        epochs=entry["epochs"],
        batch_size=entry["batch_size"],
        verbose=1,
        callbacks=callbacks
    )
    entry["history"] = history
    entry['best_model_path'] = checkpoint_path



Epoch 1/500
